## Tutorial via Example on how Langgraph Works

Langgraph allows us to have full control of the pipeline in which agents apply logic and retrys to tools etc.

In this example we are coding a deep research agent that is an expert on energy. We are using mock data.

Let's start off importing what we need.

In [9]:
!pip install langgraph

In [10]:
from typing_extensions import TypedDict
from typing import Literal
from langgraph.graph import StateGraph, START, END

### STEP 1: Define the state

The state basically is a dictionary of all the information you think each agent would need e.g. how many retries they can do, what is the max retries, what is the query, what is the retrieved documents etc. This gets updated as we go along the graph. 

In [11]:
class ResearchAgentState(TypedDict):
    question: str
    retrieved_docs: list[str]
    summary: str 
    report: str
    needs_more_info: bool

### STEP 2: Defiine the nodes 

The nodes are all the agents and are where all the tool calling and LLM generation happens. Here in our energy research agent we have the nodes retrieve_documents, summarize_documents, generate_report. At each pt at these agents/nodes the state gets updated and passed onto the next one to update.

In [12]:
def retrieve_documents(state: ResearchAgentState) -> dict:
    question = state ["question"]
    #^^ usually we would use question to simulate semantic similarity retrieval
    # Simulate retrieval of documents relevant to renewable energy
    docs = [
        "Solar energy is harnessed using photovoltaic cells.",
        "Wind energy is generated by converting wind currents into electricity.",
        "Renewable energy reduces carbon emissions and dependence on fossil. fuels."
    ]
    return {"retrieved_docs": docs, "needs_more_info": False}

def summarize_documents(state: ResearchAgentState) -> dict:
    # Simulate this summarization by concatenating the points. Pretend this is some LLM that condenses the content.
    docs = state.get ("retrieved_docs", [])
    if not isinstance(docs, list):
        raise ValueError ("Expected 'retrieved_docs' to be a list, got {type(docs)}")
    summary = " ". join(docs)
    return {"summary": summary}

def generate_report(state: ResearchAgentState) -> dict:
    question = state["question"]
    summary = state["summary"]
    # Generate a research report based on summary and question
    report = f"Research Report on: {question}\nSummary: {summary}\nConclusion: Renewable energy is vital for sustainable develópment."
    # Simulate a check if more info is needed (e.g. if summary is too short)
    needs_more_info = len(summary) < 50
    return {"report": report, "needs_more_info": needs_more_info}

### STEP 3: Define conditional routing function 

So this is after an agent/node performs a task what happens next? Which agent does it pass it too and why?

In [13]:
# After retrieve_documents
def route_after_retrieve(state: ResearchAgentState) -> Literal["summarize _documents", "retrieve_documents"]:
    if state.get("needs_more_info"): 
        return "retrieve_documents"
    return "summarize_documents"

# After generate_report
def route_after_report(state: ResearchAgentState) -> Literal["retrieve_documents", "end"]:
    if state.get("needs_more_info"):
        return "retrieve_documents"
    return "end"

### Step 4: Build graph

Nodes are where you match the function you defined up to a name which is what you call the agent. Honestly can be the same. Edges on the other hand are how you link the agents which is where the conditional routing functions become super important! Also in the edges we start with our START node which we imported and go to our END node.

TO DO LATER: Do research on conditional edges when we run things parallel (I commented it out below)

In [ ]:
# Initialise it 
builder = StateGraph(ResearchAgentState)

# Add nodes
builder.add_node("retrieve_documents", retrieve_documents)
builder.add_node("summarize_documents", summarize_documents)
builder.add_node("generate_report", generate_report)

# Define edges
builder.add_edge(START, "retrieve_documents")
builder.add_conditional_edges("retrieve_documents", 
                              route_after_retrieve, 
                              {"summarize_documents": "summarize_documents","retrieve_documents":"retrieve_documents"})
# COULD USE IF RUN PARALLEL: builder, add_conditional_edges("summarize_documents", decide_next_node, ("generate_report": "generate _report"})
builder. add_edge("summarize_documents","generate_report")
builder.add_conditional_edges("generate_report", 
                              route_after_report, 
                              {"retrieve_documents": "retrieve_documents","end": END})

# Compile the graph 
graph = builder.compile()

### STEP 5: Run the graph 

Let's run a query through

In [17]:
inputs = {"question": "What are the main renewable energy sources and their benefits?"}
result = graph.invoke(inputs)
print("Final Research Report: \n", result["report"]) 

print("\n")
print("THE STATES THROUGHOUT")
for i, output in enumerate(graph. stream(inputs, stream_mode="values"), 1):
    print (f"{1}.Current state:", output)

print("\n")
print("WHOLE RESULT")
print(result)

Final Research Report: 
 Research Report on: What are the main renewable energy sources and their benefits?
Summary: Solar energy is harnessed using photovoltaic cells. Wind energy is generated by converting wind currents into electricity. Renewable energy reduces carbon emissions and dependence on fossil. fuels.
Conclusion: Renewable energy is vital for sustainable develópment.


THE STATES THROUGHOUT
1.Current state: {'question': 'What are the main renewable energy sources and their benefits?'}
1.Current state: {'question': 'What are the main renewable energy sources and their benefits?', 'retrieved_docs': ['Solar energy is harnessed using photovoltaic cells.', 'Wind energy is generated by converting wind currents into electricity.', 'Renewable energy reduces carbon emissions and dependence on fossil. fuels.'], 'needs_more_info': False}
1.Current state: {'question': 'What are the main renewable energy sources and their benefits?', 'retrieved_docs': ['Solar energy is harnessed using p